Time Evolution

Besides ground state properties Tensor Networks can also be used to study real-time dynamics of quantum many body systems (unlike Monte Carlo). There are a couple of algorithms to do time evolution

* iTEBD
* TDVP
* Time Evolution MPO

We will focus on the latter two

In the case of quantum many body systems time evolution amounts to solving the time dependent schrodinger equation
$$i \frac{d}{dt} \ket{\Psi(t)}= \hat{H}\ket{\Psi(t)}$$
for a given Hamiltonian $\hat{H}$ with initial condition $\ket{\Psi_0}=\ket{\Psi(t_0)}$. For a time independent Hamiltonian the solution is given by $\ket{\Psi(t)} = \exp(-it\hat{H})\ket{\Psi_0}$. Constructing the exponential however is hard. For Tensor Networks the different time evolution algorithms try to implement the evolution operator approximately.

TDVP

The Time-Dependent Variational Princinple is an old idea, originaly developed by Dirac and Frenkel. The idea is to solve the schrodinger equation by minimizing
$$\|i \frac{d}{dt} \ket{\Psi(t)}- \hat{H}\ket{\Psi(t)}\|^2$$
In the case of MPS, we can parametrize the state $\ket{\Psi(t)}$ by a set of time dependent matrices $\{A_1(t),A_2(t),\dots A_N(t)\}$ (where N is the system size for finiteMPS or the size of the unit cell for infinite MPS). In other words the state $\ket{\Psi(t)}$ lives in a manifold determined by these matrices, the MPS-manifold. The solution of the minimization problem is then obtained by projecting the RHS of the schrodinger equation onto the MPS manifold 
$$\frac{d}{dt} \ket{\Psi(A)} = -i \hat{P}_{T\ket{\Psi(A)}} \hat{H}\ket{\Psi(A)}$$
where $\hat{P}_{T\ket{\Psi(A)}}$ is the operator that projects the state onto the tangent space
This means that the time-evolving state will never leave the MPS manifold and can thus be parametrized by the time dependent As. One can in principle solve the above equation by working everything out in terms the As and their time derivatives. This gives a complicated set of non-linear equation that can be solved by one's favourite finite difference scheme, but requires the inversion of matrices with small singular values (and thus numerical instabilities).

One can show that the projection operator for the mixed canonical form is given by
$$expression$$
The projection operation consists of two sums, one where we have brought the tensor on site $n$ into AC form and one where we only have the bond tensor C. The insight is that we can solve each term seperately (and in fact in any order). For each site $n$ of the MPS we have
$$\dot{A}_C(n) = -i \hat{H}_{eff}^{A_C}(n) A_C(n)$$ 
and $$\dot{C}(n) = i \hat{H}_{eff}^{C}(n) C(n)$$
Aproximating the effective Hamiltonians as independent of the other $A_C$ and $C$ these can be integrated exactly
$$A_C(n,t) = \exp(-i \hat{H}_{eff}^{A_C}(n)) A_C(n,0) $$ 
and $$C(n,t) = \exp(i \hat{H}_{eff}^{C}(n)) C(n,0) $$
A natural way to solve the set of equations is then, starting from the first site

 * update $A_C(n)$ according to the above formula
 * QR the resulting new $A_C'$ to get a new updated $A_L(n,t+dt)$ en $\tilde{C}$
 * Update the new $\tilde{C}$ via the above to get an updated $C(n+1,t+dt)$
 * Absorb the updated $C(n+1,t+dt)$ into $A_R(n+1,t)$ to get a new $A_C(n+1)$ and repeat

At the end of the chain one only updates the $A_C$ since there is no $C$ there. Doing the above left to right sweep gives a first order integrator i.e. we have solved the time evolution up to order $\mathcal{O}(dt^2)$. Since the terms can be solved in any order we can also perform a reverse sweep i.e. working from right to left. Combining this with the left to right sweep yields a second order integrator (because the reverse sweep is the adjoint of the left to right sweep).
